<h1><div align="center">비동기 스트리밍 및 CUDA C/C++를 이용한 비주얼 프로파일링</div></h1>

![CUDA](./images/CUDA_Logo.jpg)

CUDA 툴킷은 가속 CUDA 애플리케이션 개발을 지원하는 강력한 GUI 애플리케이션인 **Nsight Systems**와 함께 제공됩니다. Nsight Systems는 CUDA API 호출, 커널 실행, 메모리 활동, **CUDA 스트림** 사용에 대한 자세한 정보와 함께 가속 애플리케이션의 그래픽 타임라인을 생성합니다.

이 실습에서는 Nsight Systems의 타임라인을 사용하여 가속 애플리케이션을 최적화하는 방법을 안내합니다. 또한 **관리되지 않는 메모리 할당 및 마이그레이션**, 호스트 메모리 **고정**또는 **페이지 로킹**, **기본이 아닌 동시 CUDA 스트림** 등 작업을 지원하기 위한 몇 가지의 중급 CUDA 프로그래밍 기법을 학습합니다.

이 실습이 끝나면 여러분은 간단한 n-body 입자 시뮬레이터를 가속화 및 최적화하기 위한 평가를 거치게 되며, 이 과정을 통해 여러분이 쌓은 실력을 보여주실 수 있습니다. 시뮬레이터의 정확도를 유지하면서 가속화할 수 있는 분들께는 역량에 대한 증명으로 인증서를 수여합니다.

---
## 전제 조건

이 실습을 최대한 잘 활용하려면 여러분은 이미 다음을 하실 수 있어야 합니다.

- CPU 함수를 호출하고 GPU 커널을 실행하는 C/C++ 프로그램 작성, 컴파일, 실행
- 실행 구성을 이용한 병렬 스레드 계층 구조 제어
- 직렬 루프를 GPU에서 병렬로 반복 실행하도록 리팩터링
- CUDA 통합 메모리 할당 및 해제
- 페이지 폴트 및 데이터 마이그레이션에 관한 통합 메모리 동작 이해
- 비동기 메모리 프리페치를 사용하여 페이지 폴트 및 데이터 마이그레이션 감소

## 목표

이 실습을 완료할 때는 다음을 하실 수 있게 됩니다.

- **Nsight Systems**를 사용하여 GPU 가속 CUDA 애플리케이션의 타임라인 비주얼 프로파일링
- Nsight Systems를 사용하여 GPU 가속 CUDA 애플리케이션에서 최적화 기회 파악 및 활용
- 가속 애플리케이션에서 동시 커널 실행을 위해 CUDA 스트림 활용
- (**선택 가능 고급 콘텐츠**) 동시 CUDA 스트림에서 데이터를 비동기적으로 전송하기 위해 고정 메모리 할당을 포함한 수동 디바이스 메모리 할당 사용

---
## Nsight Systems 실행

이 대화형 실습 환경을 위해 Nsight Systems를 설치 및 사용할 수 있도록 여러분의 브라우저에서 액세스할 수 있는 원격 데스크톱을 설정했습니다.

Nsight Systems에서 이 보고서 파일을 열면 먼저 기존 벡터 추가 프로그램에 대한 보고서 파일을 생성하는 것으로 시작하여 시각적 경험을 개선하기 위한 일련의 단계를 거치게 됩니다.

### 보고서 파일 생성

[`01-vector-add.cu`](../edit/01-vector-add/01-vector-add.cu) (<-------- 이 소스 파일 링크를 클릭하여 브라우저에서 편집하세요)에는 작동하는 가속 벡터 추가 애플리케이션이 포함되어 있습니다. 바로 아래에서 코드 실행 셀을 사용하여(이것뿐 아니라 이 실습의 모든 코드 실행 셀은 `CTRL` + 클릭하면 실행할 수 있습니다)하여 컴파일 및 실행합니다. 성공했음을 나타내는 메시지가 출력되는 것을 볼 수 있습니다.

In [1]:
!nvcc -o vector-add-no-prefetch 01-vector-add/01-vector-add.cu -run

Success! All values calculated correctly.


다음으로, `nsys profile --stats=true`를 사용하여 Nsight Systems 비주얼 프로파일러에서 열 수 있는 보고서 파일을 생성합니다. 여기에서는 `-o` 플래그를 사용하여 보고서 파일에 기억하기 쉬운 이름을 지정합니다.

In [2]:
!nsys profile --stats=true -o vector-add-no-prefetch-report ./vector-add-no-prefetch


**** collection configuration ****
	output_filename = /dli/task/vector-add-no-prefetch-report
	force-overwrite = false
	stop-on-exit = true
	export_sqlite = true
	stats = true
	capture-range = none
	stop-on-range-end = false
	Beta: ftrace events:
	ftrace-keep-user-config = false
	trace-GPU-context-switch = false
	delay = 0 seconds
	duration = 0 seconds
	kill = signal number 15
	inherit-environment = true
	show-output = true
	trace-fork-before-exec = false
	sample_cpu = true
	backtrace_method = LBR
	wait = all
	trace_cublas = false
	trace_cuda = true
	trace_cudnn = false
	trace_nvtx = true
	trace_mpi = false
	trace_openacc = false
	trace_vulkan = false
	trace_opengl = true
	trace_osrt = true
	osrt-threshold = 0 nanoseconds
	cudabacktrace = false
	cudabacktrace-threshold = 0 nanoseconds
	profile_processes = tree
	application command = ./vector-add-no-prefetch
	application arguments = 
	application working directory = /dli/task
	NVTX profiler range trigger = 
	NVTX profiler domain trigge

### 원격 데스크톱 열기

다음 셀을 실행하여 원격 데스크톱 URL을 생성한 뒤 복사하여 새 브라우저 탭에 붙여넣으면 원격 데스크톱이 열립니다. 그런 다음 노트에 적힌 지침을 읽어보세요.

In [3]:
%%js
var url = window.location.hostname + '/nsight/';
element.append(url)

<IPython.core.display.Javascript object>

_Connect_ 버튼을 클릭하면 나타나는 비밀번호 란에 `nvidia`를 입력합니다.

### 원격 데스크톱 터미널 애플리케이션 열기

다음으로 원격 데스크톱 화면 하단에 있는 터미널 애플리케이션 아이콘을 클릭합니다.

![터미널](images/terminal.png)

### Nsight Systems 열기

Nsight Systems를 열려면 방금 열린 터미널에서 `nsight-sys` 명령을 입력하고 실행합니다.

![Nsight 열기](images/open-nsight.png)

### 사용 보고 활성화

메시지가 표시되면 "Yes"를 클릭하여 사용 보고를 활성화합니다.

![사용 활성화](images/enable_usage.png)

### 보고서 파일 열기

Nsight Systems 메뉴에서 _File_ -> _Open_을 선택하고 `/root/Desktop/reports` 경로로 이동한 다음 `vector-add-no-prefetch-report.qdrep`를 선택하여 보고서 파일을 엽니다. 이 실습에서 생성하는 모든 보고서는 이 `root/Desktop/reports` 디렉터리에 저장됩니다.

![보고서 열기](images/open-report.png)

### 경고/오류 무시

경고 또는 오류가 보이면 특정 원격 데스크톱 환경의 결과일 뿐이므로 무시하고 닫으셔도 됩니다.

![오류 무시](images/ignore-error.png)

### 타임라인을 위한 공간 확보

더 수월한 경험을 위해, 프로파일러를 전체 화면으로 바꾸고, _Project Explorer_를 닫고, *Events View*를 숨깁니다.

![수월하게 만들기](images/make-nice.png)

이제 다음과 같은 화면이 됩니다.

![이제 수월함](images/now-nice.png)

### CUDA 통합 메모리 타임라인 확장

다음으로, _CUDA_ -> _Unified memory_ 및 _Context_ 타임라인을 확장하고 _OS 런타임 라이브러리_ 타임라인을 닫습니다.

![메모리 열기](images/open-memory.png)

### 다수 메모리 전송 관찰

한 눈에 볼 때 애플리케이션 실행에 약 1초가 찍혔고, `addVectorsInto` 커널이 실행 중일 때 많은 UM 메모리 활동이 있습니다.

![메모리 및 커널](images/memory-and-kernel.png)

메모리 타임라인을 확대하면 온디맨드 메모리 페이지 폴트로 인한 작은 메모리 전송을 모두 더욱 명확하게 볼 수 있습니다. 몇 가지 팁:

1. `Ctrl`을 누른 채로 마우스/트랙패드를 스크롤하면 타임라인의 어느 지점이든 확대 및 축소할 수 있습니다.
2. 어느 영역이든 클릭하고 그 주변으로 드래그해 직사각형을 그린 뒤 _Zoom in_을 선택하면 확대할 수 있습니다.

다수의 작은 메모리 전송을 보기 위해 확대하는 예시입니다.

![다수의 전송](images/many-transfers.png)

---
## Nsight Systems로 반복적으로 리팩터링하는 코드 비교하기

이제 Nsight Systems을 켜서 실행해 보았고 타임라인 주변에서 움직이는 데 익숙해졌으니 여러분께 이미 익숙한 기법을 사용하여 반복적으로 개선된 일련의 프로그램을 프로파일링할 수 있습니다. 프로파일링할 때마다 타임라인의 정보가 다음번에는 어떻게 코드를 수정해야 할지 도움을 주는 정보를 제공합니다. 이렇게 함으로써 다양한 CUDA 프로그래밍 기법이 애플리케이션 성능에 어떤 영향을 미치는지 더 잘 이해할 수 있습니다.

### 연습문제: 프리페칭 타임라인과 비프리페칭 타임라인 비교

[`01-vector-add-prefetch-solution.cu`](../edit/01-vector-add/solutions/01-vector-add-prefetch-solution.cu)는 벡터 추가 애플리케이션을 리팩터링하여 `addVectorsInto` 커널이 필요로 하는 3개 벡터가 커널 실행 전 활성 GPU 디바이스에 비동기적으로 프리페칭되도록([`cudaMemPrefetchAsync`](http://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1ge8dc9199943d421bc8bc7f473df12e42) 사용) 합니다. 소스 코드를 열고 애플리케이션에서 이러한 변경 사항이 적용된 위치를 파악하세요.

변경 사항을 검토한 후 바로 아래의 코드 실행 셀을 사용하여 리팩터링된 애플리케이션을 컴파일 및 실행하세요. 성공 메시지가 출력되는 것을 볼 수 있습니다.

In [ ]:
!nvcc -o vector-add-prefetch 01-vector-add/solutions/01-vector-add-prefetch-solution.cu -run

이제 이 애플리케이션 버전에 대한 보고서 파일을 생성하세요.

In [ ]:
!nsys profile --stats=true -o vector-add-prefetch-report ./vector-add-prefetch

Nsight Systems에서 비교를 위해 이전 보고서를 열어둔 채로 보고서를 엽니다.

- 비동기 프리페칭 추가 전 `addVectorsInto` 커널과 비교했을 때 실행 시간이 어떠한가요?
- 타임라인 *CUDA API* 영역 안에 `cudaMemPrefetchAsync`를 위치시킵니다.
- 메모리 전송이 어떻게 변경되었나요?


### 연습문제: 커널 내 실행 초기화를 통한 프로파일 리팩터

벡터 추가 애플리케이션의 이전 반복에서는 벡터 데이터가 CPU에서 초기화되고 있었고, 따라서 `addVectorsInto` 커널이 작동할 수 있게 되기 전 GPU로 마이그레이션해야 합니다.

애플리케이션의 다음 반복 [01-init-kernel-solution.cu](../edit/02-init-kernel/solutions/01-init-kernel-solution.cu)에서 애플리케이션은 GPU에서 병렬로 데이터를 초기화하기 위해 리팩터링되었습니다.

이제 초기화가 GPU에서 수행되므로, 프리페칭은 벡터 추가 작업 전이 아닌 초기화 전에 이루어졌습니다. 소스 코드를 검토하여 이러한 변경 사항이 적용된 위치를 파악하세요.

변경 사항을 검토한 후 바로 아래의 코드 실행 셀을 사용하여 리팩터링된 애플리케이션을 컴파일 및 실행하세요. 성공 메시지가 출력되는 것을 볼 수 있습니다.

In [ ]:
!nvcc -o init-kernel 02-init-kernel/solutions/01-init-kernel-solution.cu -run

이제 이 애플리케이션 버전에 대한 보고서 파일을 생성하세요.

In [ ]:
!nsys profile --stats=true -o init-kernel-report ./init-kernel

Nsight Systems에서 새 보고서 파일을 열고 다음을 수행하세요.

- 애플리케이션 및 `addVectorsInto` 런타임을 이전 버전의 애플리케이션과 비교했을 때 어떻게 바뀌었나요?
- 타임라인의 *Kernel* 영역을 살펴봅시다. 두 커널(`addVectorsInto`와 초기화 커널) 중 어떤 것이 GPU에서 대부분의 시간을 소모하고 있나요?
- 다음 중 여러분의 애플리케이션에는 어떤 것이 포함되어 있습니까?
  - 데이터 마이그레이션(HtoD)
  - 데이터 마이그레이션(DtoH)

### 연습문제:호스트로 되돌아오는 비동기 프리페치로 프로파일 리팩터

현재 벡터 추가 애플리케이션은 호스트에서 벡터 추가 커널의 작업을 확인합니다. 애플리케이션의 다음 리팩터 [01-prefetch-check-solution.cu](../edit/04-prefetch-check/solutions/01-prefetch-check-solution.cu)는 데이터를 호스트로 다시 비동기 프리페치해 확인합니다.

변경 사항을 검토한 후 바로 아래의 코드 실행 셀을 사용하여 리팩터링된 애플리케이션을 컴파일 및 실행하세요. 성공 메시지가 출력되는 것을 볼 수 있습니다.

In [ ]:
!nvcc -o prefetch-to-host 04-prefetch-check/solutions/01-prefetch-check-solution.cu -run

이제 이 애플리케이션 버전에 대한 보고서 파일을 생성하세요.

In [ ]:
!nsys profile --stats=true -o prefetch-to-host-report ./prefetch-to-host

Nsight Systems에서 이 보고서 파일을 열고 다음을 수행하세요.

- 타임라인의 *Unified Memory* 영역을 사용하여 CPU로의 프리페칭을 추가하기 전과 추가한 후 *데이터 마이그레이션(DtoH)* 이벤트를 비교 및 대조하세요.

---
## 동시 CUDA 스트림

이제 **CUDA 스트림**이라는 새로운 컨셉에 대해 알아볼 것입니다. 개론을 배우고 나면 Nsight Systems 사용으로 돌아와 스트림이 애플리케이션 성능에 미치는 영향을 더 잘 평가할 수 있습니다.

다음 슬라이드에서는 앞으로 다룰 콘텐츠를 고수준에서 시각적으로 보여드립니다. 다음 섹션으로 넘어가 각 주제를 더 자세히 다루기 전에 슬라이드를 클릭해 보세요.

In [ ]:
%%HTML

<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vR-tYNce--gUNXExn_KU29u9EldZY4Jo2R_f9fccxulPH_Y5S_bUrG5Y8qyxsUuuyC8cRs9gQbo9ZTu/embed?start=false&loop=false&delayms=3000" frameborder="0" width="900" height="550" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

CUDA 프로그래밍에서 **스트림**이란 순서대로 실행되는 일련의 명령입니다. CUDA 애플리케이션에서는 커널 실행과 일부 메모리 전송이 CUDA 스트림 내에서 일어납니다. 이 시점까지의 콘텐츠에서 여러분은 CUDA 스트림과 명시적으로 상호작용하지는 않았지만, 여러분의 CUDA 코드는 *기본 스트림*이라고 부르는 스트림 내에서 커널을 실행하고 있었습니다.

CUDA 프로그래머는 기본 스트림 외에 기본이 아닌 CUDA 스트림을 생성하고 활용할 수 있으며 이를 통해 여러 개의 커널을 동시에 다른 스트림에서 실행하는 등 여러 연산을 수행할 수 있습니다. 여러 개의 스트림을 사용하면 가속 애플리케이션에 병렬 처리 계층을 추가할 수 있으며 애플리케이션 최적화를 위한 더 많은 기회를 제공합니다.

### CUDA 스트림의 동작을 지배하는 규칙

CUDA 스트림의 동작에 대한 몇 가지 규칙이 있으며, 이를 효과적으로 활용하기 위해서는 학습해야 합니다.

- 주어진 스트림 내의 연산은 순서대로 이루어집니다.
- 기본이 아닌 서로 다른 스트림에서의 연산은 서로 상대적인 특정 순서로 작동하도록 보장되지 않습니다.
- 기본 스트림은 차단되어 실행 전 다른 모든 스트림이 완료될 때까지 기다릴 것이고, 이 스트림이 완료될 때까지 다른 스트림이 실행되지 못하게 차단합니다.

### 기본이 아닌 CUDA 스트림 생성, 활용, 폐기

다음 코드 조각은 기본이 아닌 CUDA 스트림을 생성, 활용, 폐기하는 방법을 시연합니다. 기본이 아닌 CUDA 스트림에서 CUDA 커널을 실행하려면 스트림이 실행 구성의 선택적 네 번째 인수로 전달되어야 한다는 것을 알 수 있습니다. 지금까지는 실행 구성의 처음 2개 인수만 활용했습니다.

```cpp
cudaStream_t stream;       // CUDA streams are of type `cudaStream_t`.
cudaStreamCreate(&stream); // Note that a pointer must be passed to `cudaCreateStream`.

someKernel<<<number_of_blocks, threads_per_block, 0, stream>>>(); // `stream` is passed as 4th EC argument.

cudaStreamDestroy(stream); // Note that a value, not a pointer, is passed to `cudaDestroyStream`.
```

이 실습의 범위를 벗어났지만 언급할 가치가 있는 것은 실행 구성에 대한 선택적 세 번째 인수입니다. 이 인수를 통해 프로그래머는 **공유 메모리**(지금은 다루지 않을 고급 주제)의 바이트 수를 이 커널 실행에 대해 블록당 동적으로 할당하도록 제공합니다. 블록당 공유 메모리에 할당된 기본 바이트의 수는 `0`이고, 나머지 실습에서 여러분은 당장의 관심사인 네 번째 인수를 노출하기 위해 이 값으로 `0`을 전달하게 됩니다.

### 연습문제: 기본 스트림 동작 예측

[01-print-numbers](../edit/05-stream-intro/01-print-numbers.cu) 애플리케이션에는 정수를 받아들여 출력하는 매우 간단한 `printNumber` 커널이 있습니다. 커널은 단일 블록 내부의 단일 스레드로만 실행되고 있지만 for 루프를 사용하여 5번 실행되고 있으며 각 실행에 for 루프의 반복 횟수를 전달합니다.

아래의 코드 실행 블록을 사용하여 [01-print-numbers](../edit/05-stream-intro/01-print-numbers.cu)을 컴파일 및 실행하세요. `0`부터 `4`까지의 숫자가 출력되는 것을 볼 수 있습니다.

In [ ]:
!nvcc -o print-numbers 05-stream-intro/01-print-numbers.cu -run

기본적으로 커널이 기본 스트림에서 실행된다는 것을 알고 있을 때, `print-numbers` 프로그램의 5회 실행이 직렬과 병렬 중 어떻게 실행될 것이라고 예상하시나요? 답변을 뒷받침하기 위해서는 기본 스트림의 두 가지 기능을 언급할 수 있어야 합니다. 아래 셀에서 보고서 파일을 생성하고 Nsight Systems에서 열어 답변을 확인하세요.

In [ ]:
!nsys profile --stats=true -o print-numbers-report ./print-numbers

### 연습문제: 동시 CUDA 스트림 구현

5개 커널 실행이 모두 동일한 스트림에서 발생하기 때문에 5개 커널이 직렬로 실행되는 것을 보아도 놀라지는 않으실 것입니다. 또한 기본 스트림이 차단되어 있기 때문에 커널의 각 실행이 다음 실행 전에 완료될 때까지 기다릴 것이라고 주장할 수도 있으며, 이 역시 참입니다.

[01-print-numbers](../edit/05-stream-intro/01-print-numbers.cu)을 리팩터링하여 각 커널 실행이 기본이 아닌 자체 스트림에서 발생하게 하세요. 생성한 스트림이 필요 없어지면 반드시 폐기하세요. 바로 아래의 코드 실행 셀을 사용하여 리팩터링된 코드를 컴파일 및 실행하세요. 여전히 `0`부터 `4`까지의 숫자가 출력되는 것을 볼 수 있지만, 반드시 오름차순은 아닙니다. 도중에 막히면 [해답](../edit/05-stream-intro/solutions/01-print-numbers-solution.cu)을 참고하세요.

In [ ]:
!nvcc -o print-numbers-in-streams 05-stream-intro/01-print-numbers.cu -run

각 5개 커널 실행에 대해 기본이 아닌 5개의 다른 스트림을 사용 중인 지금, 직렬과 병렬 중 어떻게 실행될 것이라고 예상하시나요? 현재 스트림에 대해 알고 있는 것 외에도 `printNumber` 커널이 얼마나 사소한지를 고려하세요. 즉, 병렬 실행을 예측하는 경우에도 하나의 커널이 완료되는 속도가 완전한 중첩이 가능한 수준일까요?

가설을 세운 후 Nsight Systems에서 새 보고서 파일을 열어 실제 동작을 볼 수 있습니다. 이제 _CUDA_ 영역에 여러분이 생성한 기본이 아닌 각 스트림에 해당하는 추가 줄이 있는 것을 알 수 있습니다.

In [ ]:
!nsys profile --stats=true -o print-numbers-in-streams-report print-numbers-in-streams

![스트림 중첩](images/streams-overlap.png)

### 연습문제: 동시 데이터 초기화 커널에 스트림 사용

여러분이 작업하고 있는 벡터 추가 애플리케이션 [01-prefetch-check-solution.cu](../edit/04-prefetch-check/solutions/01-prefetch-check-solution.cu)는 현재 초기화 커널을 3번, 즉 `vectorAdd` 커널에 대해 초기화가 필요한 3개 벡터에 각각 한 번씩 실행합니다. 3개 초기화 커널 각각이 기본이 아닌 자체 스트림에서 실행되도록 리팩터링하세요. 아래 코드 실행 셀을 사용하여 컴파일 및 실행 시 여전히 성공 메시지가 출력되는 것이 보여야 합니다. 도중에 막히면 [해답](../edit/06-stream-init/solutions/01-stream-init-solution.cu)을 참고하세요.

In [ ]:
!nvcc -o init-in-streams 04-prefetch-check/solutions/01-prefetch-check-solution.cu -run

Nsight Systems에서 보고서를 열어 3번의 초기화 커널 실행이 기본이 아닌 자체 스트림에서 실행되고 어느 정도의 동시 중첩이 있는지 확인할 수 있습니다.

In [ ]:
!nsys profile --stats=true -o init-in-streams-report ./init-in-streams

---
## 요약

이 시점의 실습에서는 다음을 할 수 있습니다.

- **Nsight Systems**를 사용하여 GPU 가속 CUDA 애플리케이션의 타임라인 비주얼 프로파일링
- Nsight Systems를 사용하여 GPU 가속 CUDA 애플리케이션에서 최적화 기회 파악 및 활용
- 가속 애플리케이션에서 동시 커널 실행을 위해 CUDA 스트림 활용

이 시점에서 여러분은 CPU 전용 애플리케이션을 가속화하고 이러한 가속 애플리케이션을 최적화하기 위한 풍부한 기본 툴과 기법을 갖췄습니다. 마지막 연습문제에서는 학습한 모든 것을 적용하여 개체 그룹이 중력으로 상호 작용할 때의 개별 움직임을 예측하는 [n-body](https://en.wikipedia.org/wiki/N-body_problem) 시뮬레이터를 가속할 기회를 얻게 됩니다.

---
## 최종 연습문제: N-Body 시뮬레이터 가속화 및 최적화

[n-body](https://en.wikipedia.org/wiki/N-body_problem) 시뮬레이터는 개체 그룹이 중력으로 상호 작용할 때의 개별 움직임을 예측합니다. [01-nbody.cu](../edit/09-nbody/01-nbody.cu)에는 3차원 공간을 통해 움직이는 물체를 위한 단순하지만 작동하는 n-body 시뮬레이터가 포함되어 있습니다.

이 애플리케이션은 현재 CPU 전용 형태로, 4096개 입자에서 실행되는 데 약 5초, 65536개 입자에서 실행되는 데는 **20분**이 소요됩니다. 여러분이 할 일은 시뮬레이션의 정확도를 유지하면서 프로그램을 GPU 가속하는 것입니다.

### 작업 안내를 위한 고려 사항

작업을 시작하기 전에 고려해야 할 몇 가지 사항이 있습니다.

- 특히 처음 몇 번의 리팩터의 경우 애플리케이션의 로직, 특히 `bodyForce` 함수가 크게 변하지 않은 상태로 유지되어야 합니다. 가능한 한 쉽게 가속화하는 데 집중하세요.
- 코드베이스 `main` 내에는 `bodyForce`로 계산된 물체 간의 힘을 시스템 내 물체 위치로 통합하기 위한 for 루프가 포함되어 있습니다. 이 통합은 `bodyForce` 실행 다음에 이루어져야 하며, 다음 `bodyForce` 호출 전에 완료되어야 합니다. 병렬 처리 방법과 위치를 선택할 때 이를 염두에 두세요.
- **프로파일 중심**의 반복적 접근법을 사용하세요.
- 코드에 오류 처리를 추가할 필요는 없지만 여러분이 코드가 올바르게 작동할 책임을 져야 하므로 추가해 두면 유용할 수 있습니다.

**즐겁게 작업해 보세요!**

이 셀을 사용하여 nbody 시뮬레이터를 컴파일하세요. 초기에는 CPU 전용 애플리케이션이지만, 입자의 위치를 정확하게 시뮬레이션합니다.

In [ ]:
!nvcc -std=c++11 -o nbody 09-nbody/01-nbody.cu

프로파일러를 사용하여 작업에 도움을 받으실 것을 권장합니다. 다음 셀을 실행하여 보고서 파일을 생성하세요.

In [ ]:
!nsys profile --stats=true --force-overwrite=true -o nbody-report ./nbody

여기에서는 여러분의 `nbody` 시뮬레이터를 다양한 입자 수에 대해 실행시키는 함수를 불러와 성능과 정확도를 검사합니다.

In [ ]:
from assessment import run_assessment

`nbody` 실행 및 평가를 위해 다음 셀을 실행하세요.

In [ ]:
run_assessment()

## 인증서 생성

평가를 통과한 경우 과정 페이지(아래 그림)로 돌아가서 과정에 대한 인증서를 생성하는 "ASSESS TASK" 버튼을 클릭합니다.

![평가 실행](./images/run_assessment.png)

## 고급 콘텐츠

다음 섹션에서는 시간과 관심이 있는 분들을 위해 몇 가지 수동 디바이스 메모리와 관련되어 있거나 기본이 아닌 스트림을 사용하여 커널 실행과 메모리 사본을 중첩시키는 더 많은 중급 기법을 소개합니다.

아래의 각 기법에 대해 학습한 후 이러한 기법을 사용하여 nbody 시뮬레이션을 더욱 최적화해 보세요.

---
## 수동 디바이스 메모리 할당 및 복사

`cudaMallocManaged` 및 `cudaMemPrefetchAsync`는 성능이 우수하고 메모리 마이그레이션을 매우 간소화하는 반면, 가끔은 메모리 할당을 위해 수동 방법을 더 많이 사용하는 것이 나을 수도 있습니다. 이는 데이터가 디바이스 또는 호스트에서만 액세스되는 것이 알려져 있는 경우와 자동 온디맨드 마이그레이션이 필요 없다는 사실에 대한 대가로 데이터 마이그레이션 비용을 재구성할 수 있는 경우 특히 참입니다.

또한 수동 디바이스 메모리 관리를 사용하면 연산 작업으로 기본이 아닌 스트림을 데이터 전송 중첩에 사용할 수 있습니다. 이 섹션에서는 이러한 기법을 연산 작업으로 데이터 사본을 중첩하는 데까지 확장하기 전에 몇 가지 기본 수동 디바이스 메모리 할당 및 복사 기법을 학습합니다. 

수동 디바이스 메모리 관리를 위한 몇 가지 CUDA 명령입니다.

- `cudaMalloc`은 메모리를 활성 GPU에 직접 할당합니다. 따라서 모든 GPU 페이지 폴트를 방지할 수 있습니다. 그 대신, 반환되는 포인터는 호스트 코드로 액세스할 수 없습니다.
- `cudaMallocHost`는 메모리를 CPU에 직접 할당합니다. 또한 메모리를 "고정"하거나 페이지 로킹하여 GPU를 오가며 메모리를 비동기 복사할 수 있도록 해 줍니다. 고정 메모리가 너무 많으면 CPU 성능을 방해할 수 있으므로 의도가 있을 때만 사용하세요. 고정 메모리는 `cudaFreeHost`로 해제할 수 있습니다.
- `cudaMemcpy`는 호스트에서 디바이스 또는 디바이스에서 호스트로 메모리를 복사(전송 아님)할 수 있습니다.

### 수동 디바이스 메모리 관리 예시

다음은 위 CUDA API 호출의 사용을 시연하는 코드 조각입니다.

```cpp
int *host_a, *device_a;        // Define host-specific and device-specific arrays.
cudaMalloc(&device_a, size);   // `device_a` is immediately available on the GPU.
cudaMallocHost(&host_a, size); // `host_a` is immediately available on CPU, and is page-locked, or pinned.

initializeOnHost(host_a, N);   // No CPU page faulting since memory is already allocated on the host.

// `cudaMemcpy` takes the destination, source, size, and a CUDA-provided variable for the direction of the copy.
cudaMemcpy(device_a, host_a, size, cudaMemcpyHostToDevice);

kernel<<<blocks, threads, 0, someStream>>>(device_a, N);

// `cudaMemcpy` can also copy data from device to host.
cudaMemcpy(host_a, device_a, size, cudaMemcpyDeviceToHost);

verifyOnHost(host_a, N);

cudaFree(device_a);
cudaFreeHost(host_a);          // Free pinned memory like this.
```

### 연습문제: 호스트 및 디바이스 메모리 수동으로 할당

벡터 추가 애플리케이션의 최신 반복인 [01-stream-init-solution](../edit/06-stream-init/solutions/01-stream-init-solution.cu)은 `cudaMallocManaged`를 사용하여 디바이스에서 처음 사용된 관리되는 메모리를 먼저 초기화 커널로 할당한 다음, 벡터 추가 커널로 디바이스에 할당하고, 메모리가 자동으로 전송되는 호스트로 할당하여 확인합니다. 이 방식이 합리적이긴 하지만, 애플리케이션의 성능에 미치는 영향을 관찰하기 위해 수동 디바이스 메모리 할당 및 복사를 약간 실험해 보는 것도 가치 있는 일입니다.

[01-stream-init-solution](../edit/06-stream-init/solutions/01-stream-init-solution.cu) 애플리케이션이 `cudaMallocManaged`를 사용하지 **않도록** 리팩터링하세요. 이를 위해서는 다음을 해야 합니다.

- `cudaMallocManaged`로의 호출을 `cudaMalloc`으로 교체하세요.
- 호스트에서 확인에 사용될 추가 벡터를 생성하세요. `cudaMalloc`에 할당된 메모리를 호스트에서 사용할 수 없기 때문에 필요합니다. 이 호스트 벡터를 `cudaMallocHost`에 할당하세요.
- `addVectorsInto` 커널이 완료된 후 `cudaMemcpy`를 사용하여 `cudaMallocHost`로 생성한 호스트 벡터로 추가 결과와 함께 벡터를 복사할 수 있습니다.
- `cudaMallocHost`로 할당된 메모리를 해제하려면 `cudaFreeHost`를 사용하세요.

도중에 막히면 [해답](../edit/07-manual-malloc/solutions/01-manual-malloc-solution.cu)을 참고하세요.

In [ ]:
!nvcc -o vector-add-manual-alloc 06-stream-init/solutions/01-stream-init-solution.cu -run

리팩터를 완료한 후 Nsight Systems에서 보고서를 열고 타임라인을 사용하여 다음에 따르세요.

- 타임라인의 *Unified Memory* 영역이 더 이상 없다는 것을 알 수 있습니다.
- 이 타임라인을 이전 리팩터의 타임라인과 비교하면서 현재 애플리케이션의 `cudaMalloc` 런타임과 이전 애플리케이션의 `cudaMallocManaged`의 런타임을 비교하세요.
- 현재 애플리케이션에서 초기화 커널의 작업이 이전 반복에서 시작했던 시간 이후에 시작된다는 것을 알 수 있습니다. 타임라인을 검토하면 `cudaMallocHost`에 소요된 시간의 차이를 볼 수 있습니다. 이를 통해 메모리 전송과 메모리 복사의 차이를 분명하게 알 수 있습니다. 지금 하고 있는 것처럼 메모리를 복사할 때는 데이터가 시스템의 서로 다른 2개 장소에 존재합니다. 현재의 경우 네 번째 호스트 전용 벡터의 할당은 이전 반복에서 3개 벡터만 할당할 때에 비해 성능 면에서 적은 비용을 발생시킵니다.

---
## 스트림을 사용하여 데이터 전송 및 코드 실행 중첩하기

다음 슬라이드에서는 앞으로 다룰 콘텐츠를 고수준에서 시각적으로 보여드립니다. 다음 섹션으로 넘어가 각 주제를 더 자세히 다루기 전에 슬라이드를 클릭해 보세요.

In [ ]:
%%HTML

<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vRhw6_OQ3oY57IIgs8JH2AgZpP9gNJ87ee3IYE3RV8Ox7CLj2RLPAm2YtxyhjfDDBjC6owOOfpKL1JS/embed?start=false&loop=false&delayms=3000" frameborder="0" width="900" height="550" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

`cudaMemcpy` 말고도 `cudaMemcpyAsync` 역시 `cudaMallocHost`로 할당하여 호스트 메모리가 고정되어 있는 한 호스트에서 디바이스 또는 디바이스에서 호스트로 메모리를 비동기 복사할 수 있습니다.

커널 실행과 유사하게 `cudaMemcpyAsync`은 호스트에 대해 기본적으로 비동기입니다. 이것은 기본적으로 기본 스트림에서 실행되므로 따라서 GPU에서 발생하는 다른 CUDA 연산에 대한 차단 연산입니다. 그러나 `cudaMemcpyAsync` 함수는 선택적 다섯 번째 인수인 기본이 아닌 스트림으로 사용됩니다. 이 스트림을 기본이 아닌 스트림으로 전달하면 메모리 전송이 기본이 아닌 다른 스트림에서 발생하는 다른 CUDA 연산과 동시에 이루어질 수 있습니다.

흔히 사용되는 유용한 패턴은 고정 호스트 메모리, 기본이 아닌 스트림에서의 비동기 메모리 사본, 기본이 아닌 스트림에서의 커널 실행을 결합하여 메모리 전송을 커널 실행과 중첩하는 것입니다.

다음 예시에서는 커널 작업을 시작하기 전에 전체 메모리 복사가 완료될 때까지 기다리는 대신 필요한 데이터 세그먼트가 복사되고 작동하며, 각 복사/작업 세그먼트가 기본이 아닌 자체 스트림에서 실행됩니다. 이 기법을 사용하면 데이터 파트에 대한 작업을 이후 세그먼트에 대한 메모리 전송 발생과 동시에 시작할 수 있습니다. 이 기법을 사용하여 연산 수 및 어레이 내부의 오프셋 위치에 대한 세그먼트별 값을 계산할 때는 특히 주의해야 합니다.

```cpp
int N = 2<<24;
int size = N * sizeof(int);

int *host_array;
int *device_array;

cudaMallocHost(&host_array, size);               // Pinned host memory allocation.
cudaMalloc(&device_array, size);                 // Allocation directly on the active GPU device.

initializeData(host_array, N);                   // Assume this application needs to initialize on the host.

const int numberOfSegments = 4;                  // This example demonstrates slicing the work into 4 segments.
int segmentN = N / numberOfSegments;             // A value for a segment's worth of `N` is needed.
size_t segmentSize = size / numberOfSegments;    // A value for a segment's worth of `size` is needed.

// For each of the 4 segments...
for (int i = 0; i < numberOfSegments; ++i)
{
  // Calculate the index where this particular segment should operate within the larger arrays.
  segmentOffset = i * segmentN;

  // Create a stream for this segment's worth of copy and work.
  cudaStream_t stream;
  cudaStreamCreate(&stream);
  
  // Asynchronously copy segment's worth of pinned host memory to device over non-default stream.
  cudaMemcpyAsync(&device_array[segmentOffset],  // Take care to access correct location in array.
                  &host_array[segmentOffset],    // Take care to access correct location in array.
                  segmentSize,                   // Only copy a segment's worth of memory.
                  cudaMemcpyHostToDevice,
                  stream);                       // Provide optional argument for non-default stream.
                  
  // Execute segment's worth of work over same non-default stream as memory copy.
  kernel<<<number_of_blocks, threads_per_block, 0, stream>>>(&device_array[segmentOffset], segmentN);
  
  // `cudaStreamDestroy` will return immediately (is non-blocking), but will not actually destroy stream until
  // all stream operations are complete.
  cudaStreamDestroy(stream);
}
```

### 연습문제: 커널 실행 및 메모리 사본을 호스트로 다시 중첩하기

벡터 추가 애플리케이션의 최신 반복 [01-manual-malloc-solution.cu](../edit/07-manual-malloc/solutions/01-manual-malloc-solution.cu)는 현재 검증을 위해 메모리를 호스트로 다시 복사하기 전에 GPU에서 모든 벡터 추가 작업을 수행하고 있습니다.

[01-manual-malloc-solution.cu](../edit/07-manual-malloc/solutions/01-manual-malloc-solution.cu)를 리팩터링하여 비동기 메모리 사본이 모든 벡터 추가 작업을 완료하기 전에 시작할 수 있도록 4개 세그먼트에서 벡터 추가를 수행하세요. 도중에 막히면 [해답](../edit/08-overlap-xfer/solutions/01-overlap-xfer-solution.cu)을 참고하세요.

In [ ]:
!nvcc -o vector-add-manual-alloc 07-manual-malloc/solutions/01-manual-malloc-solution.cu -run

리팩터를 완료한 후 Nsight Systems에서 보고서를 열고 타임라인을 사용하여 다음에 따르세요.

- 디바이스에서 호스트로 메모리 전송이 시작될 때 모든 커널 작업을 완료하기 전인지 후인지 확인하셨나요?
- 4개 메모리 사본 세그먼트 자체가 서로 중첩되지 않음을 알 수 있습니다. 기본이 아닌 별도의 스트림에서도 주어진 방향(여기서는 DtoH)으로는 한 번의 메모리 전송만 동시에 발생할 수 있습니다. 여기에서의 성능 향상은 그렇게 하지 않았을 때보다 전송을 더 일찍 시작하는 능력에 있으며, 단순한 추가 작업에 비해 더 적은 양의 작업이 수행되는 애플리케이션에서는 메모리 복사가 더 일찍 시작될 뿐만 아니라 커널 실행과 중첩된다는 것을 상상하기 어렵지 않습니다.